In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

In [2]:
train = pd.read_csv('train.csv',parse_dates=['fecha']).drop(columns=['lat','lng'])
test = pd.read_csv('test.csv',parse_dates=['fecha']).drop(columns=['lat','lng'])

In [3]:
#Arreglo problema de metros totales y metros cubiertos
train['metrostotales_reales'] = train['metrostotales'].where(train.metrostotales>=train.metroscubiertos,train.metroscubiertos)
train.metroscubiertos = train['metroscubiertos'].where(train.metrostotales>=train.metroscubiertos,train.metrostotales)
train.metrostotales = train.metrostotales_reales
train.drop(columns=['metrostotales_reales'],inplace=True)

In [4]:
#Arreglo problema de metros totales y metros cubiertos
test['metrostotales_reales'] = test['metrostotales'].where(test.metrostotales>=test.metroscubiertos,test.metroscubiertos)
test.metroscubiertos = test['metroscubiertos'].where(test.metrostotales>=test.metroscubiertos,test.metrostotales)
test.metrostotales = test.metrostotales_reales
test.drop(columns=['metrostotales_reales'],inplace=True)

In [5]:
train.tipodepropiedad.isnull().sum()

46

In [6]:
cat_features = ['tipodepropiedad', 'provincia', 'ciudad']
num_features = ['banos','garages','habitaciones','metrostotales','metroscubiertos','antiguedad']
bool_features = ['gimnasio','usosmultiples','piscina','escuelascercanas','centroscomercialescercanos']

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [8]:
transformador_categorias = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
transformador_numeros = SimpleImputer()

In [9]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformador_numeros, num_features),
        ('cat', transformador_categorias, cat_features)
    ])

In [10]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=2000, learning_rate=0.05, max_depth=10, subsample_by_tree=0.5, colsample_by_col=0.5, min_child_weight=3)
from catboost import CatBoostRegressor
model2 = CatBoostRegressor(iterations=1000, depth=10, learning_rate=0.05, loss_function='RMSE', task_type="GPU")

In [11]:
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])
pipeline2 = Pipeline(steps =[('preprocessor', preprocessor),
                             ('model', model2)])

In [12]:
features = ['tipodepropiedad', 'antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 'provincia','ciudad'] 
train_y = train.precio.copy()
train_x = train[features].copy()

In [ ]:
my_pipeline.fit(train_x, train_y)